In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/gender/Training/"
test_dir = "/content/drive/MyDrive/gender/Validation/"


# Using imagedatagenerator to access the files and image augmentation.

# You can change the setting of imageDataGenerator for other augmentation methods.

train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)


training_generator = train_datagen.flow_from_directory(train_dir,
                                                      target_size=(224, 224),
                                                      class_mode="binary")


test_generator = test_datagen.flow_from_directory(test_dir,
                                                       target_size=(224, 224),
                                                       class_mode="binary")

Found 22667 images belonging to 2 classes.
Found 5472 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
import keras
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
target_size = (224, 224)
batch_size = 128
mode = 'binary'

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg.trainable = False

In [ ]:
model_with_tuning = 'transfer_model.h5'
checkpoint_callback = ModelCheckpoint(model_with_tuning,
                                     monitor='val_accuracy',
                                     save_best_only=True,
                                     verbose=1)
earlystop = EarlyStopping(monitor='val_accuracy',patience=5,verbose=1)

In [ ]:
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
             metrics=['accuracy'],
             optimizer='adam')

In [ ]:
history_without_tuning = model.fit(training_generator,
                   steps_per_epoch=22667  // batch_size,
                   epochs=10,
                   validation_data=test_generator,
                   validation_steps=5472 // batch_size,
                   verbose=1,
                   callbacks = [checkpoint_callback,earlystop])

Epoch 1/10
177/177 [==============================] - 2565s 14s/step - loss: 0.7228 - accuracy: 0.7874 - val_loss: 0.2150 - val_accuracy: 0.9144

Epoch 00001: val_accuracy improved from -inf to 0.91443, saving model to transfer_model.h5
Epoch 2/10
177/177 [==============================] - 2043s 12s/step - loss: 0.1820 - accuracy: 0.9315 - val_loss: 0.1603 - val_accuracy: 0.9345

Epoch 00002: val_accuracy improved from 0.91443 to 0.93452, saving model to transfer_model.h5
Epoch 3/10
177/177 [==============================] - 1403s 8s/step - loss: 0.1509 - accuracy: 0.9462 - val_loss: 0.1724 - val_accuracy: 0.9390

Epoch 00003: val_accuracy improved from 0.93452 to 0.93899, saving model to transfer_model.h5
Epoch 4/10
177/177 [==============================] - 1177s 7s/step - loss: 0.1521 - accuracy: 0.9440 - val_loss: 0.2075 - val_accuracy: 0.9182

Epoch 00004: val_accuracy did not improve from 0.93899
Epoch 5/10
177/177 [==============================] - 790s 4s/step - loss: 0.1621 - 

In [ ]:
model.save('/content/drive/MyDrive/transfer_model.h5')
#model = keras.models.load_model('/content/drive/MyDrive/transfer_model_model.h5')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test = "/content/drive/MyDrive/pic/"


test_data = ImageDataGenerator(rescale=1/255)


test_generator = test_data.flow_from_directory(test,
                                              target_size=(224, 224),
                                              class_mode="binary")

Found 20 images belonging to 2 classes.


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('test acc:', test_acc)
print('test_loss:',test_loss)

1/1 [==============================] - 1s 1s/step - loss: 0.0034 - accuracy: 1.0000
test acc: 1.0
test_loss: 0.003373892279341817
